In [1]:
%load_ext autoreload
%autoreload 2
from tqdm.notebook import tqdm
from dataloader import *
import os
import numpy as np
from concurrent.futures import ThreadPoolExecutor

def write_batch(batch, batch_nr, batch_size, path):
    curr = 0
    items = batch.items()
    num_samples_in_batch = batch["audio"].numpy().shape[0]
    for i in range(num_samples_in_batch):
        dct = {k:v[i] for k,v in items}
        np.savez(os.path.join(path,"{}.npz".format(batch_nr*batch_size+curr)), **dct)
        curr += 1

def create_numpy_sets(override = False):
    batch_size = 256
    datasets = make_datasets_original(batch_size, True, True, percent=100)
    names = ["train","validation","test"]
    sizes = [60788, 17469, 8518]
    for ds,path,size in zip(datasets,names,sizes):
        print("Preparing",path)
        batch_nr = 0
        path = os.path.join("data",path)
        if override and os.path.exists(path):
            os.system("rm -rf {}".format(path))
        os.makedirs(path,exist_ok=True)
       
        for batch in tqdm(iter(ds), total=np.ceil(size/batch_size)):
            # write batch
            with ThreadPoolExecutor(max_workers=5) as executor:
                executor.submit(write_batch, batch, batch_nr, batch_size, path)
                batch_nr+=1

In [2]:
#create_numpy_sets(override=True)

In [8]:
train_set,val_set,test_set = make_datasets(64)

In [10]:
import time
start = time.time()
for batch in train_set.take(10):
    continue
stop = time.time()
print((stop-start)*100/60)

7.194952170054118
